<a href="https://colab.research.google.com/github/wqiu96/summer_project/blob/master/src/Deeplearning_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! git clone https://github.com/frankhan91/DeepBSDE.git

Cloning into 'DeepBSDE'...
remote: Enumerating objects: 20, done.
remote: Total 20 (delta 0), reused 0 (delta 0), pack-reused 20
Unpacking objects: 100% (20/20), done.


In [0]:
cd DeepBSDE/

/content/DeepBSDE


In [0]:
import json
import logging
import os
import numpy as np
import tensorflow as tf
from config import get_config
from equation import get_equation
from solver import FeedForwardModel

FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')
tf.app.flags.DEFINE_string('problem_name', 'HJB',
                           """The name of partial differential equation.""")
tf.app.flags.DEFINE_integer('num_run', 1,
                            """The number of experiments to repeatedly run for the same problem.""")
tf.app.flags.DEFINE_string('log_dir', './logs',
                           """Directory where to write event logs and output array.""")


def main():
    problem_name = FLAGS.problem_name
    config = get_config(problem_name)
    bsde = get_equation(problem_name, config.dim, config.total_time, config.num_time_interval)

    if not os.path.exists(FLAGS.log_dir):
        os.mkdir(FLAGS.log_dir)
    path_prefix = os.path.join(FLAGS.log_dir, problem_name)
    with open('{}_config.json'.format(path_prefix), 'w') as outfile:
        json.dump(dict((name, getattr(config, name))
                       for name in dir(config) if not name.startswith('__')),
                  outfile, indent=2)
    logging.basicConfig(level=logging.INFO,
                        format='%(levelname)-6s %(message)s')

    for idx_run in range(1, FLAGS.num_run+1):
        tf.reset_default_graph()
        with tf.Session() as sess:
            logging.info('Begin to solve %s with run %d' % (problem_name, idx_run))
            model = FeedForwardModel(config, bsde, sess)
            if bsde.y_init:
                logging.info('Y0_true: %.4e' % bsde.y_init)
            model.build()
            training_history = model.train()
            if bsde.y_init:
                logging.info('relative error of Y0: %s',
                             '{:.2%}'.format(
                                 abs(bsde.y_init - training_history[-1, 2])/bsde.y_init))
            # save training history
            np.savetxt('{}_training_history_{}.csv'.format(path_prefix, idx_run),
                       training_history,
                       fmt=['%d', '%.5e', '%.5e', '%d'],
                       delimiter=",",
                       header="step,loss_function,target_value,elapsed_time",
                       comments='')

if __name__ == '__main__':
    main()

INFO   Begin to solve HJB with run 1


Instructions for updating:
Colocations handled automatically by placer.


WARNING From /usr/local/lib/python2.7/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
keep_dims is deprecated, use keepdims instead


WARNING From equation.py:98: calling reduce_sum_v1 (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
Use tf.cast instead.


WARNING From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
INFO   step:     0,    loss: 1.9327e+01,   Y0: 1.9596e-01,  elapsed time  23
INFO   step:   100,    loss: 5.7515e+00,   Y0: 1.1026e+00,  elapsed time  39
INFO   step:   200,    loss: 2.7857e+00,   Y0: 1.4134e+00,  elapsed time  46
INFO   step:   300,    loss: 2.5864e+00,   Y0: 1.6258e+00,  elapsed time  53
INFO   step:   400,    loss: 2.3271e+00,   Y0: 1.8679e+00,  elapsed time  60
INFO   step:   500,    loss: 2.0594e+00,   Y0: 2.1397e+00,  elapsed time  67
INFO   step:   600,    loss: 1.8110e+00,   Y0: 2.4377e+00,  elapsed time  74
INFO   step:   700,    loss: 1.5003e+00,   Y0: 2.7700e+00,  elapsed time  81
INFO   step:   800,    loss: 1.2043e+00,   Y0: 3.1083e+00,  elapsed time  88
INFO   step:   900,    loss: 8.5866e-01,   Y0: 3.4630e+00,  elaps